In [4]:
import pandas as pd
from ta.trend import SMAIndicator, MACD
from ta.momentum import RSIIndicator

# read the first two rows to get column names
with open("crypto_4.csv", "r") as file:
    price_row = file.readline().strip().split(",")[1:]  # Skip the first empty cell
    ticker_row = file.readline().strip().split(",")[1:]  # Skip "Ticker"

# Create proper column names by combining Price and Ticker rows
column_names = [f"{price} {ticker}" for price, ticker in zip(price_row, ticker_row)]

# Now load the data, starting from the Date row
data = pd.read_csv("crypto_4.csv", skiprows=2, index_col=0, parse_dates=True)
data.columns = column_names  # Assign the correct column names
print("Column names in crypto_4.csv after renaming:")
print(data.columns.tolist())

# Define tickers
tickers = ["BNB-USD", "BTC-USD", "ETH-USD", "XRP-USD"]

# Process each ticker separately
for ticker in tickers:
    # Select columns for this ticker
    df = data[[f"Open {ticker}", f"High {ticker}", f"Low {ticker}", 
               f"Close {ticker}", f"Volume {ticker}"]].copy()
    df.columns = ["Open", "High", "Low", "Close", "Volume"]  # Rename for simplicity
    
    # Clean: Fill missing values and drop NaNs
    df.ffill(inplace=True)
    df.dropna(inplace=True)
    
    # Feature Engineering
    df["Lag1"] = df["Close"].shift(1)
    df["Lag7"] = df["Close"].shift(7)
    df["SMA7"] = SMAIndicator(df["Close"], window=7).sma_indicator()
    df["RSI14"] = RSIIndicator(df["Close"], window=14).rsi()
    df["MACD"] = MACD(df["Close"], window_slow=26, window_fast=12).macd()
    df["Returns"] = df["Close"].pct_change()
    df["Volatility"] = df["High"] - df["Low"]
    df["Target"] = df["Close"].shift(-1)
    
    
    # Drop rows with NaN 
    df.dropna(inplace=True)
    
    # Save processed data
    df.to_csv(f"{ticker}_processed.csv")
    print(f"Processed {ticker}: {df.shape} rows, columns: {list(df.columns)}")


print("\nBTC-USD Sample:")
print(pd.read_csv("BTC-USD_processed.csv").tail())

Column names in crypto_4.csv after renaming:
['Close BNB-USD', 'Close BTC-USD', 'Close ETH-USD', 'Close XRP-USD', 'High BNB-USD', 'High BTC-USD', 'High ETH-USD', 'High XRP-USD', 'Low BNB-USD', 'Low BTC-USD', 'Low ETH-USD', 'Low XRP-USD', 'Open BNB-USD', 'Open BTC-USD', 'Open ETH-USD', 'Open XRP-USD', 'Volume BNB-USD', 'Volume BTC-USD', 'Volume ETH-USD', 'Volume XRP-USD']
Processed BNB-USD: (705, 13) rows, columns: ['Open', 'High', 'Low', 'Close', 'Volume', 'Lag1', 'Lag7', 'SMA7', 'RSI14', 'MACD', 'Returns', 'Target', 'Volatility']
Processed BTC-USD: (705, 13) rows, columns: ['Open', 'High', 'Low', 'Close', 'Volume', 'Lag1', 'Lag7', 'SMA7', 'RSI14', 'MACD', 'Returns', 'Target', 'Volatility']
Processed ETH-USD: (705, 13) rows, columns: ['Open', 'High', 'Low', 'Close', 'Volume', 'Lag1', 'Lag7', 'SMA7', 'RSI14', 'MACD', 'Returns', 'Target', 'Volatility']
Processed XRP-USD: (705, 13) rows, columns: ['Open', 'High', 'Low', 'Close', 'Volume', 'Lag1', 'Lag7', 'SMA7', 'RSI14', 'MACD', 'Returns'